<a href="https://colab.research.google.com/github/jiw3026/baseline_/blob/main/04_3_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC(%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%EB%AA%A8%EB%8D%B8%EB%A7%81%ED%8F%AC%ED%95%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 전처리
- 전처리 기법 : 토큰화, 노이즈/불용어 제거, 통일화, 품사태깅, 벡터화
- 위의 전처리 기법이 실제 분석에서 어떻게 이루어지는지 파악하고 익혀보자

## konlpy 설치

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 45.7 MB/s eta 0:00:00


In [ ]:
# 이미 앞서 다뤄본 적이 있는 기본적인 라이브러리
import numpy as np
import pandas as pd

import json
import re

from tqdm.notebook import tqdm

In [ ]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

#### 형태소 분석기별 메서드 정리 
출처 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/

---
1) Hannanum : KAIST에서 개발   
- analyze : 다양한 형태학적 후보를 반환
- morphs : 형태소 토큰화 
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---  
2) Kkma : 서울대에서 개발
- morphs : 형태소 토큰화 
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환 
- sentences : 문장 토큰화
---  
3) Komoran : Shineware팀에서 개발
- morphs : 형태소 토큰화 
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---
4) Mecab : 교토대에서 일본어용으로 개발된 분석기, 은전 프로젝트에 의해 한국어용 개발
- morphs : 형태소 토큰화 
- nouns : 명사 토큰만 반환
- pos : 형태소에 품사 태깅하여 반환
---
5) Okt(Twitter) : Will Hohyon Ryu가 개발
- morphs : 형태소 토큰화 
- nouns : 명사 토큰만 반환
- phrases : 다양한 형태학적 후보를 반환
- pos : 형태소에 품사 태깅하여 반환


In [ ]:
import os
from datetime import datetime

## Download Dataset(github에서 NSMC 데이터셋 다운로드)

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [ ]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [ ]:
display(train.head())
display(test.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
train.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [ ]:
test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
X_train = train.document   
y_train = train.label 

X_test = test.document  
y_test = test.label 
X_train[0], y_train[0]

('아 더빙.. 진짜 짜증나네요 목소리', 0)

In [ ]:
print('#Train set size:', len(X_train))
print('#Test set size:', len(X_test))

#Train set size: 149995
#Test set size: 49997


## 토큰화 + 노이즈/불용어제거 + 통일화 + 주요 품사 추출(품사태깅)

### 토큰화

In [ ]:
review = X_train[0]

In [ ]:
okt = Okt()

In [ ]:
result = okt.morphs(review) 
result

['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

### 노이즈 제거

In [ ]:
# 한글제외한 나머지 문자 제거 --> 정규표현식
sub_noise = lambda text : re.sub('[^가-힣]', '', text)
result = [sub_noise(text) for text in result]
result

['아', '더빙', '', '진짜', '짜증나네요', '목소리']

In [ ]:
# 1글자짜리 제거 
result = [text for text in result if len(text) > 1]
result

['더빙', '진짜', '짜증나네요', '목소리']

### 불용어 제거

In [ ]:
# 불용어 사전 - 리스트로 사용
stopwords = ['목소리']

with open('nsmc_stopwords.txt', 'wt') as f:
    f.write('\n'.join(stopwords))  # 리스트 -> 문자열

# f = open('nsmc_stopwords.txt', 'wt')
# f.write(stopwords_str)
# f.close()

In [ ]:
f = open('nsmc_stopwords.txt')
stopwords = f.read().split()
stopwords

['목소리']

In [ ]:
result = [token for token in result if not token in stopwords]
result

['더빙', '진짜', '짜증나네요']

### 품사태깅

In [ ]:
# 품사 태깅의 이해
review = X_train[0]
review

'아 더빙.. 진짜 짜증나네요 목소리'

In [ ]:
review_text = re.sub("[^가-힣\\s]", "", review)
review_text

'아 더빙 진짜 짜증나네요 목소리'

In [ ]:
okt = Okt()
word_review = okt.pos(review_text, stem=True)
word_review

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나다', 'Adjective'),
 ('목소리', 'Noun')]

In [ ]:
for token, pos in word_review:
    print(token, pos)

아 Exclamation
더빙 Noun
진짜 Noun
짜증나다 Adjective
목소리 Noun


### 종합적인 구현

In [ ]:
f = open('nsmc_stopwords.txt') # 경로 + 파일명
stop_words = f.read().split()
stop_words

['목소리']

In [ ]:
# 1. 한글 및 공백을 제외한 문자 모두 제거.
import re
review_text = re.sub("[^가-힣\s]", "", '아.. 더빙 진짜 짜증나네')
review_text

'아 더빙 진짜 짜증나네'

In [ ]:
okt = Okt()
word_review = okt.pos(review_text, stem=True)
word_review

[('아', 'Exclamation'), ('더빙', 'Noun'), ('진짜', 'Noun'), ('짜증나다', 'Adjective')]

In [ ]:
# pos 결과 이해하기
## 1. word_review에서 '더빙'만 출력해보자.
word_review[1][0]

## 2. 품사들만 출력해보자.
for word in word_review:
    print(word[1])

## 3. 토큰은 tokens에, 품사는 pos에 리스트로 각각 담아보자.
tokens, pos = [], []

for word in word_review:
    tokens.append(word[0])
    pos.append(word[1])
tokens, pos

Exclamation
Noun
Noun
Adjective


(['아', '더빙', '진짜', '짜증나다'], ['Exclamation', 'Noun', 'Noun', 'Adjective'])

In [ ]:
def preprocessing(review): 
    okt = Okt()
    
    f = open('nsmc_stopwords.txt')
    stop_words = f.read().split()
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_review = okt.pos(review_text, stem=True)
    
    # 노이즈 & 불용어 제거
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token) > 1]
    
    # 명사, 동사, 형용사 추출
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]

    return word_review

In [ ]:
for x in X_train[:100]:
    print(preprocessing(x))

['더빙', '진짜', '짜증나다']
['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍다', '않다']
['무재', '밓었', '다그', '래서', '보다', '추천']
['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']
['몬페', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '늙다', '보이다', '하다', '커스틴', '던스트', '이쁘다', '보이다']
['걸음', '떼다', '초등학교', '학년', '생인', '살다', '영화', '반개', '아깝다']
['원작', '긴장감', '제대로', '살리다', '하다']
['반개', '아깝다', '나오다', '이응경', '길용우', '생활', '정말', '해도', '그것', '낫다', '납치', '감금', '반복', '반복', '드라마', '가족', '없다', '연기', '하다', '사람']
['액션', '없다', '재미', '있다', '안되다', '영화']
['왜케', '평점', '낮다', '헐리우드', '화려하다', '길들이다', '있다']
['인피니트', '진짜']
['볼때', '눈물나다', '죽다', '년대', '향수', '허진호', '감성', '절제', '멜로', '달인']
['울면', '횡단보도', '건너다', '뛰다', '치다', '이범수', '연기', '드럽다', '하다']
['담백하다', '깔끔하다', '좋다', '로만', '보다', '보다', '자꾸', '잊어버리다', '사람', '이다']
['취향', '존중', '다지', '진짜', '내생', '극장', '보다', '영화', '가장', '노잼', '감동', '스토리', '어거지', '감동', '어거지']
['매번', '긴장', '되다', '재밋음']
['차다', '사람', '웃기다', '바스코', '이기', '락스', '끄다', '바비', '이기', '아이돌', '깔다', '다그', '끄다', '안달', '보이다']
['굿바이', '레닌', '표절',

### 벡터화 
1. BoW : CountVectorizer
2. TF-IDF : TfidfVectorizer

In [ ]:
CountVectorizer?

Object `CountVectorizer` not found.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

bow = CountVectorizer(tokenizer=preprocessing, min_df=5, max_df=0.5) 

# X_train_bow = bow.fit(X_train)
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=2000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 모델 학습 및 검증

In [ ]:
# 모델 1 : Logistic Regression 모형
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train_bow, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train_bow, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test_bow, y_test)))

Train set score: 0.850
Test set score: 0.819


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 모델 2 : 트리 앙상블 모형
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators = 100, max_depth=30, random_state = 0)
model_rf.fit(X_train_bow, y_train)
print('Train set score: {:.3f}'.format(model_rf.score(X_train_bow, y_train)))
print('Test set score: {:.3f}'.format(model_rf.score(X_test_bow, y_test)))

Train set score: 0.791
Test set score: 0.770


In [ ]:
# 모델 3 : lightgbm Model
import lightgbm as lgb

model_lgbm = lgb.LGBMClassifier()
model_lgbm.fit(X_train_bow, y_train)
print('Train set score: {:.3f}'.format(model_lgbm.score(X_train_bow, y_train)))
print('Test set score: {:.3f}'.format(model_lgbm.score(X_test_bow, y_test)))

In [ ]:
clf = model_lgbm

for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)

## 모델 학습 및 검증

In [ ]:
# 모델 1 : Logistic Regression 모형
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test_tfidf, y_test)))

In [ ]:
# 모델 2 : 트리 앙상블 모형
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators = 100, max_depth=30, random_state = 0)
model_rf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(model_rf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(model_rf.score(X_test_tfidf, y_test)))

In [ ]:
# 모델 3 : lightgbm Model
import lightgbm as lgb

model_lgbm = lgb.LGBMClassifier()
model_lgbm.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(model_lgbm.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(model_lgbm.score(X_test_tfidf, y_test)))

In [ ]:
clf = model_lgbm

for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)